Calculates discounted cashflows for Fixed-Floating and OIS (TRAINING DATA)

In [2]:
# 1. Imports
import os
import pandas as pd
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


# Paths
#r'C:/Users/gusta/Documents/KTH/TriOptima/trioptima/trioptima/'
#'/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/'

# 2. Configuration
your_path = r'C:/Users/gusta/Documents/KTH/TriOptima/trioptima/trioptima/'
train_folder_path = your_path + '6.Active Data/Train Model Data/'
test_folder_path = your_path + '6.Active Data/Test Data/'

# 3. Functions
def convert_to_usd(row,exchange_rates):
    return row['leg1NotionalAmount'] * exchange_rates.get(row['leg1NotionalCurrency'], 1)

def calculate_cashflows(row, period, freq):
    cashflows = []
    start_date = row['effectiveDate']
    end_date = row['expirationDate']
    
    if row[period] == 'MNTH':
        delta = relativedelta(months=row[freq])
    elif row[period] == 'YEAR':
        delta = relativedelta(years=row[freq])
    elif row[period] == 'DAIL':
        delta = timedelta(days=row[freq])
    else:
        raise ValueError(f"Unknown frequency period: {row[period]}")
    
    while start_date <= end_date:
        cashflows.append(start_date)
        start_date += delta

    return cashflows

def convert_to_years(value):
    
    if "M" in value:
        return int(str(value)[:-1])/12
    elif "Y" in value:
        return int(str(value)[:-1])
    else:
        return 0

def floating_rate(years_to_payout, float_rates_df, bump):
    differences = float_rates_df['Tenor'].sub(years_to_payout).abs()
    nearest_index = differences.idxmin()
    return (float_rates_df.loc[nearest_index, 'Rate'] + bump)/ 100

def calculate_discounted_cashflow(row, rate_type, bump, float_rates_df_list):
    total_discounted_cashflow = 0
    
    # CHECK
    discounted_cashflow_list = []
    cashflow_list = []
    factors = []
    pr_list = []
    discount_rate = []

    ibor = row['leg2UnderlierID']
    for item in float_rates_df_list:
        if item[0] == ibor:
            float_rates_df = item[1]
            break 
    
    for date in row['cashflow_dates']:

        if len(cashflow_list) == 0:

            time_difference = (date - datetime.now()).days / 365.0
            tenor = time_difference ### MONTHS IS TO MATCH THE INDEX IN OUR IBOR FILE
            float_rate = floating_rate(tenor,float_rates_df, bump)
            factor = (1 + float_rate)**time_difference
            factors.append(factor)
            discount_rate.append([tenor,1/ factor])

            if rate_type == 'float':
                payout_rate = float_rate
                cashflow = row['leg1NotionalAmount'] * payout_rate
                
            if rate_type == 'fixed':
                payout_rate = row['leg1FixedRate']
                cashflow = row['leg1NotionalAmount'] * payout_rate
                
            discounted_cashflow = cashflow / factor
        
        else:
            time_difference = ((date-row['cashflow_dates'][len(factors)-1])).days/365
            tenor = (date - datetime.now()).days / 365
            float_rate = floating_rate(tenor,float_rates_df,bump)
            prev_factor = factors[-1]
            factor = prev_factor * (1 + payout_rate) ** time_difference
            factors.append(factor)
            discount_rate.append([tenor,1/ factor])
            if rate_type == 'float':    
                payout_rate = float_rate
                cashflow = row['leg1NotionalAmount'] * payout_rate
            
            if rate_type == 'fixed':
                payout_rate = row['leg1FixedRate']
                cashflow = row['leg1NotionalAmount'] * row['leg1FixedRate']
                
            discounted_cashflow = cashflow / factor
        pr_list.append(payout_rate)
        total_discounted_cashflow += discounted_cashflow
        
        # CHECK
        cashflow_list.append(cashflow)
        discounted_cashflow_list.append(discounted_cashflow)
    
    # CHECK 
    #print('Total discounted CF '+rate_type+': '+str(total_discounted_cashflow))
    #print(rate_type + ' cashflow list ' + str((cashflow_list)))
    #print(rate_type + ' '+ibor+' discount rates ' + str((discount_rate)))
    #print('\n')
    
    #print(rate_type + ' discounted cashflow list ' + str(discounted_cashflow_list) )
    #print(rate_type + ' discount factors ' + str((factors)))
    #print('\n')
    return total_discounted_cashflow

def fx_rates (your_path):
    exchange_rates_df = pd.read_csv(your_path + '7.IBOR/exchange_rates.csv')
    exchange_rates = dict(zip(exchange_rates_df['Currency'], exchange_rates_df['Rate_to_USD']))
    return exchange_rates

def fl_df (your_path, ibor):
    ibor_df = pd.read_csv(your_path + '7.IBOR/' + ibor + '.csv')
    ibor_df['Tenor'] = ibor_df['Tenor'].apply(convert_to_years) ## MAKE SURE ALL FILES ARE CODED THE SAME
    float_rates_df = ibor_df[['Tenor', 'Rate']].copy()
    return float_rates_df

def main (folder_path, your_path, bump):
    # 4. Data Loading
    # Load trade data
    files = [f for f in os.listdir(folder_path) if f != '.DS_Store']
    data_file = files[0][:-4]
    data = pd.read_csv(your_path + "2.Cleaned/" + data_file +'_Cleaned.csv')

    # Load exchange rates
    fx_df = fx_rates(your_path)
   
    # 5. Data Transformation and Filtering
    filtered_data = data[
        (data['leg1NotionalCurrency'].isin(['EUR', 'USD', 'GBP', 'AUD', 'CAD', 'NZD'])) & 
        (data['leg1UnderlyingAssetOrContractType'].isin(['Fixed-Floating', 'OIS']))
    ]
    df = pd.DataFrame(filtered_data)
    df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
    df['expirationDate'] = pd.to_datetime(df['expirationDate'])

    df['cashflow_dates'] = df.apply(
        lambda row: 
            calculate_cashflows(row, 'leg1FixedRatePaymentFrequencyPeriod', 'leg1FixedRatePaymentFrequencyMultiplier') 
            if row['leg1UnderlyingAssetOrContractType'] == 'Fixed-Floating' 
            else (calculate_cashflows(row, 'leg2UnderlierTenorPeriod', 'leg2UnderlierTenorMultiplier') 
                if row['leg1UnderlyingAssetOrContractType'] == 'OIS' 
                else None), 
        axis=1)

    # 6. Based on data - Load relevant IBORs to be used as float_rates in operations
    float_rates_df_list = []
    for ibor in df['leg2UnderlierID'].unique():
        float_rates_df_list.append([ibor,fl_df(your_path,ibor)])
        
    # 7. Main Operations
    df['leg1NotionalAmountUSD'] = df.apply(lambda row: convert_to_usd(row, fx_df), axis=1)
    df['MtM_leg1'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'fixed', 0, float_rates_df_list), axis=1)
    df['MtM_leg2'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'float', 0, float_rates_df_list), axis=1)
    df['MtM_leg2_bumped'] = df.apply(lambda row: calculate_discounted_cashflow(row, 'float', bump/100, float_rates_df_list), axis=1)
    df['total_delta'] = (df['MtM_leg2'] - df['MtM_leg2_bumped']).abs()
    
    # Creates outliers in the test data if the test file is IRS_2023_ValueTest 
    print(df['MtM_leg1'])
    if data_file == 'IRS_2023_ValueTest':
        i = 0
        print('OUTLIERS')
        for value in df['MtM_leg1']:
            
            df.at[i, 'MtM_leg1'] = value * random.randint(1,50) / 10000
            i += 1
    print(df['MtM_leg1'])
    # 8. Export/Output
    df.drop(columns=['cashflow_dates', 'MtM_leg2_bumped'], inplace=True)
    df.to_csv(your_path + '3.Cash_Risk/' + data_file + '_Cash_Risk.csv', index=False)

# folder path, your path, bump (basis points, used to measure risk), outlier (used to create outliers in test data)
main(train_folder_path, your_path,bump=1)
main(test_folder_path, your_path,bump=1)


C:\Users\gusta\AppData\Local\Temp\ipykernel_6816\1522823947.py:155: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['effectiveDate'] = pd.to_datetime(df['effectiveDate'])
C:\Users\gusta\AppData\Local\Temp\ipykernel_6816\1522823947.py:156: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['expirationDate'] = pd.to_datetime(df['expirationDate'])


0       6.899811e+06
1       2.671650e+07
2       5.150168e+06
3       6.899811e+06
4       5.307547e+04
            ...     
4965    1.766709e+06
4968    7.403697e+06
4969    6.825405e+06
4970    1.541897e+07
4971    1.374079e+07
Name: MtM_leg1, Length: 3362, dtype: float64
0       6.899811e+06
1       2.671650e+07
2       5.150168e+06
3       6.899811e+06
4       5.307547e+04
            ...     
4965    1.766709e+06
4968    7.403697e+06
4969    6.825405e+06
4970    1.541897e+07
4971    1.374079e+07
Name: MtM_leg1, Length: 3362, dtype: float64
0     6.026333e+06
1     9.683671e+09
2     5.444166e+06
3     1.712699e+06
4     1.302658e+06
5     2.833611e+06
6     5.806084e+05
7     2.324708e+07
8     1.023817e+07
9     3.531587e+09
10    3.390008e+09
11    1.045771e+07
12    1.553185e+08
13    5.655136e+08
14    7.568616e+08
15    1.107458e+09
16    1.453129e+10
17    1.845885e+08
18    1.905144e+07
19    7.149071e+05
20    2.487358e+06
21    4.357316e+07
22    2.728127e+08
23    8.068